# Gastric Warriors Differential gene expression analysis



Importing correct files: Using bash


In [1]:
%%bash

mkdir -p Data/
workdir="/cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/Data/"
echo "Working directory set to $workdir"

RNAlist="/cephyr/NOBACKUP/groups/bbt045_2025/Projects/Data/RNA-seq-list.txt"
RNAdata="/cephyr/NOBACKUP/groups/bbt045_2025/Projects/Data/RNAseq/"
echo "RNAseq-data in ${RNAdata}"


grep "Met" $RNAlist | cut -f 1 > patients.txt
grep "Hp" $RNAlist | cut -f 1 >> patients.txt

find $RNAdata -type f -name "*_1.fastq.gz" | grep -Ff patients.txt | xargs -I {} cp {} "$workdir"
find $RNAdata -type f -name "*_2.fastq.gz" | grep -Ff patients.txt | xargs -I {} cp {} "$workdir"

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


Working directory set to /cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/Data/
RNAseq-data in /cephyr/NOBACKUP/groups/bbt045_2025/Projects/Data/RNAseq/


Create a CSV with patient list and health condition

In [ ]:
%%bash

#Create patient list
RNAlist="/cephyr/NOBACKUP/groups/bbt045_2025/Projects/Data/RNA-seq-list.txt"

# Define the output file
OUTPUT="patient_list.csv"
# Add CSV header
echo "PatientNumber,HealthCondition" > $OUTPUT

# Create patient_list.txt with "Met" and "Hp" conditions
awk '{print $1, ",Met"}' <(grep "Met" $RNAlist | cut -f1) >> $OUTPUT
awk '{print $1, ",Hp"}' <(grep "Hp" $RNAlist | cut -f1) >> $OUTPUT


Now decompress all the files

In [2]:
%%bash
#set the working directory
workdir="/cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/Data/"

#use gzip to decrompress the files
gzip -d "${workdir}"*

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


# MultiQC

In [1]:
#First generate fastQC reports, all files in Data and output to /subset_fastqc/
! mkdir -p before_trim_fastqc
! fastqc Data/*.fastq -o before_trim_fastqc/

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory
/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory
null
null
Started analysis of ERR950168_1.fastq
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
Approx 5% complete for ERR950168_1.fastq
Approx 10% complete for ERR950168_1.fastq
Approx 15% complete for ERR950168_1.fastq
Approx 20% complete for ERR950168_1.fastq
Approx 25% complete for ERR950168_1.fastq
Approx 30% complete for ERR950168_1.fastq
Approx 35% complete for ERR950168_1.fastq
Approx 40% complete for ERR950168_1.fastq
Approx 45% complete for ERR950168_1.fastq
Approx 50% complete for ERR950168_1.fastq
Approx 55% complete for ERR950168_1.fastq
Approx 60% complete for ERR950168_1.fastq
Approx 65% complete for ERR950168_1.fastq
Approx 70% complete for ERR950168_1.fastq
Approx 75% complete for ERR950168_1

In [2]:
#Take entire subset_fastqc/ and generate multiqc-file
! multiqc before_trim_fastqc/ -n multiqc_before_trim.html

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory

/// ]8;id=566247;https://multiqc.info\MultiQC]8;;\ 🔍 v1.27.1

       file_search | Search path: /cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/before_trim_fastqc
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 36/36  stqc/ERR950187_2_fastqc.html.html
            fastqc | Found 18 reports
     write_results | Data        : multiqc_before_trim_data
     write_results | Report      : multiqc_before_trim.html
           multiqc | MultiQC complete


# Preprocessing using FastP
> 

In [3]:
%%bash

#Output
mkdir -p trimmed_fastq


# Loop through all forward reads (_R1.fastq.gz.subset.fastq) and find their corresponding reverse reads (_R2.fastq.gz.subset.fastq)
for R1 in Data/*_1.fastq; do
    # Extract the sample name by removing _R1.fastq.gz.subset.fastq
    SAMPLE=$(basename "$R1" _1.fastq)
    # Define file names
    R2="Data/${SAMPLE}_2.fastq"         # Reverse read file

    echo "Checking: $R1 and $R2"

    OUT_R1="trimmed_fastq/${SAMPLE}_1_trimmed_fastq.gz"
    OUT_R2="trimmed_fastq/${SAMPLE}_2_trimmed_fastq.gz"
    # Run fastp, input for -i and -I (Forward and Reverse reads), output into OUT_ R1 and R2. Filter scores under 30, and better trimming with cut_right
    fastp -i "$R1" -I "$R2" -o "$OUT_R1" -O "$OUT_R2" \
        --qualified_quality_phred 30 \
        --cut_right \
        --cut_right_window_size 4 \
        --cut_right_mean_quality 30 \
        --cut_front 

    echo "Finished processing: $SAMPLE"
done

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


Checking: Data/ERR950168_1.fastq and Data/ERR950168_2.fastq


Read1 before filtering:
total reads: 24913290
total bases: 2516242290
Q20 bases: 2354746125(93.5819%)
Q30 bases: 2141558358(85.1094%)

Read2 before filtering:
total reads: 24913290
total bases: 2516242290
Q20 bases: 2355137804(93.5974%)
Q30 bases: 2196623898(87.2978%)

Read1 after filtering:
total reads: 19493030
total bases: 1538088835
Q20 bases: 1536952633(99.9261%)
Q30 bases: 1511522404(98.2728%)

Read2 after filtering:
total reads: 19493030
total bases: 1596778475
Q20 bases: 1595365240(99.9115%)
Q30 bases: 1574038254(98.5759%)

Filtering result:
reads passed filter: 38986060
reads failed due to low quality: 44620
reads failed due to too many N: 0
reads failed due to too short: 10795900
reads with adapter trimmed: 5938208
bases trimmed due to adapters: 119420244

Duplication rate: 29.7874%

Insert size peak (evaluated by paired-end reads): 101

JSON report: fastp.json
HTML report: fastp.html

fastp -i Data/ERR950168_1.fastq -I Data/ERR950168_2.fastq -o trimmed_fastq/ERR950168_1_trim

Finished processing: ERR950168
Checking: Data/ERR950169_1.fastq and Data/ERR950169_2.fastq


Read1 before filtering:
total reads: 33285817
total bases: 3361867517
Q20 bases: 3026738306(90.0315%)
Q30 bases: 2691689990(80.0653%)

Read2 before filtering:
total reads: 33285817
total bases: 3361867517
Q20 bases: 3084045147(91.7361%)
Q30 bases: 2844590197(84.6134%)

Read1 after filtering:
total reads: 20859542
total bases: 1563426927
Q20 bases: 1562271896(99.9261%)
Q30 bases: 1535629009(98.222%)

Read2 after filtering:
total reads: 20859542
total bases: 1630666733
Q20 bases: 1628899215(99.8916%)
Q30 bases: 1604600902(98.4015%)

Filtering result:
reads passed filter: 41719084
reads failed due to low quality: 131232
reads failed due to too many N: 0
reads failed due to too short: 24721318
reads with adapter trimmed: 14139482
bases trimmed due to adapters: 456404715

Duplication rate: 30.6051%

Insert size peak (evaluated by paired-end reads): 1

JSON report: fastp.json
HTML report: fastp.html

fastp -i Data/ERR950169_1.fastq -I Data/ERR950169_2.fastq -o trimmed_fastq/ERR950169_1_trimm

...

Finished processing: ERR950185
Checking: Data/ERR950186_1.fastq and Data/ERR950186_2.fastq


Read1 before filtering:
total reads: 27746271
total bases: 2802373371
Q20 bases: 2560099890(91.3547%)
Q30 bases: 2279462928(81.3404%)

Read2 before filtering:
total reads: 27746271
total bases: 2802373371
Q20 bases: 2551204755(91.0373%)
Q30 bases: 2327482393(83.054%)

Read1 after filtering:
total reads: 17218529
total bases: 1333657967
Q20 bases: 1332631753(99.9231%)
Q30 bases: 1308762605(98.1333%)

Read2 after filtering:
total reads: 17218529
total bases: 1360785871
Q20 bases: 1359363668(99.8955%)
Q30 bases: 1337664046(98.3008%)

Filtering result:
reads passed filter: 34437058
reads failed due to low quality: 121082
reads failed due to too many N: 0
reads failed due to too short: 20934402
reads with adapter trimmed: 8541264
bases trimmed due to adapters: 273963332

Duplication rate: 24.145%

Insert size peak (evaluated by paired-end reads): 1

JSON report: fastp.json
HTML report: fastp.html

fastp -i Data/ERR950186_1.fastq -I Data/ERR950186_2.fastq -o trimmed_fastq/ERR950186_1_trimmed

Finished processing: ERR950186
Checking: Data/ERR950187_1.fastq and Data/ERR950187_2.fastq


Read1 before filtering:
total reads: 25368011
total bases: 2562169111
Q20 bases: 2384491410(93.0653%)
Q30 bases: 2156338195(84.1607%)

Read2 before filtering:
total reads: 25368011
total bases: 2562169111
Q20 bases: 2370739198(92.5286%)
Q30 bases: 2198227216(85.7956%)

Read1 after filtering:
total reads: 19729754
total bases: 1535952891
Q20 bases: 1534890689(99.9308%)
Q30 bases: 1508901083(98.2388%)

Read2 after filtering:
total reads: 19729754
total bases: 1603006056
Q20 bases: 1601524474(99.9076%)
Q30 bases: 1579316882(98.5222%)

Filtering result:
reads passed filter: 39459508
reads failed due to low quality: 45774
reads failed due to too many N: 0
reads failed due to too short: 11230740
reads with adapter trimmed: 4463866
bases trimmed due to adapters: 87357795

Duplication rate: 36.8104%

Insert size peak (evaluated by paired-end reads): 1

JSON report: fastp.json
HTML report: fastp.html

fastp -i Data/ERR950187_1.fastq -I Data/ERR950187_2.fastq -o trimmed_fastq/ERR950187_1_trimmed

Finished processing: ERR950187


# FastQC and multiQC on processed reads

In [5]:
#Fastqc of the fastp dataset
! mkdir -p FastQC_aftertrim
! fastqc trimmed_fastq/*_fastq.gz -o FastQC_aftertrim/
! multiqc FastQC_aftertrim/ -n aftertrim_multiqc.html

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory
/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory
application/gzip
application/gzip
Started analysis of ERR950168_1_trimmed_fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Approx 5% complete for ERR950168_1_trimmed_fastq.gz
Approx 10% complete for ERR950168_1_trimmed_fastq.gz
Approx 15% complete for ERR950168_1_trimmed_fastq.gz
Approx 20% complete for ERR950168_1_trimmed_fastq.gz
Approx 25% complete for ERR950168_1_trimmed_fastq.gz
Approx 30% complete for ERR950168_1_trimmed_fastq.gz
Approx 35% complete for ERR950168_1_trimmed_fastq.gz
Approx 40% complete for E

# Building STAR reference geneome
Genome from https://www.ensembl.org/Homo_sapiens/Info/Index 

In [ ]:
%%bash

gunzip Reference/*.gz 

mkdir -p human_index

STAR --runMode genomeGenerate --genomeDir human_index/ --genomeFastaFiles Reference/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa \
    --sjdbGTFfile Reference/Homo_sapiens.GRCh38.113.gtf --runThreadN 8

In [18]:
import pandas as pd
#df = pd.read_csv("gene_counts.txt", sep="\t", comment="#")
patient_list = pd.read_csv("patient_list.csv")

NameError: name 'PatientNumber' is not defined

In [5]:
import pandas as pd

# Load featureCounts file, skipping header lines (lines starting with "#")
gene_counts_file = "gene_counts.txt"
df = pd.read_csv(gene_counts_file, sep="\t", comment="#")

# Rename the columns by extracting patient names from file paths
df.columns = df.columns.str.replace(r'.*/JOB_TMP/(\w+)_Aligned.sortedByCoord.out.bam', r'\1', regex=True)

# Set Geneid as index
df.set_index("Geneid", inplace=True)


df.to_csv("gene_counts_cleaned.csv")

counts = df.drop(columns=['Chr','Start','End','Length','Strand'])

# Save as CSV for later use
counts.to_csv("counts_data.csv")

# Display the first few row
print(counts.head())


                 ERR950168  ERR950169  ERR950170  ERR950177  ERR950178  \
Geneid                                                                   
ENSG00000142611        600        640        447        706        984   
ENSG00000284616          0          0          0          0          0   
ENSG00000157911         45        113         45         56         25   
ENSG00000260972          0          0          0          0          0   
ENSG00000224340          0          0          0          3          4   

                 ERR950179  ERR950185  ERR950186  ERR950187  
Geneid                                                       
ENSG00000142611        874        345        765        704  
ENSG00000284616          0          0          0          0  
ENSG00000157911         27         57         32         57  
ENSG00000260972          0          0          0          0  
ENSG00000224340          1          0          2          2  
